# Overview

This is an implementation of a neural network as a class with training and prediction capabilities. 

In [1]:
import numpy as np
import random
import copy
import scipy.io

In [93]:
'''
Neural network capable of both entropy loss and squared loss
'''
class NeuralNetwork(object):
    
    def __init__(self, inputLayerSize, hiddenLayerSize, outputLayerSize, loss_func_string):
    
        # Assign neural network loss and activiation functions
        if loss_func_string == "entropy":
            self.cost = self.entropyLoss
            self.costPrime = self.entropyLossPrime
        else:
            self.cost = self.costSquaredLoss
            self.costPrime = self.costSquaredLossPrime
        # In the future these can be user specified
        self.hiddenLayerActFunc = np.tanh
        self.hiddenLayerActFuncPrime = self.tanhPrime 
        self.outputLayerActFunc = self.sigmoid
        self.outputLayerActFuncPrime = self.sigmoidPrime
            
        # Layer dimensions
        self.inputLayerSize=inputLayerSize # plus bias
        self.hiddenLayerSize=hiddenLayerSize # plus bias
        self.outputLayerSize=outputLayerSize
        
        # In the future these can be user specified
        self.eta = None
        np.random.seed(324)
        self.w1=0.01*np.random.randn(self.inputLayerSize+1, self.hiddenLayerSize+1)
        self.w2=0.01*np.random.randn(self.hiddenLayerSize+1, self.outputLayerSize)
        
        
        # Document the training progress
        self.training_accuracy = []
        self.validation_accuracy = []
        self.trainingTime = []
        
        # Current iteration variables
        self.z1 = None
        self.y1 = None
        self.z2 = None
        self.h = None # Y before one hot encoding
        self.y = None # One hot encoding 
        self.dEdz1 = None
        self.dEdz2 = None
        self.Ytrain = None # Actual digits
        self.Ytrain_encoded = None # One hot encoding
        self.Xtrain = None # data

    # Squared loss
    def costSquaredLoss(self):
        J=0.5*np.sum(np.square(self.y-self.h))
        return J
    
    # Squared loss derivative
    def costSquaredLossPrime(self):
        dJdh = -(self.y-self.h)
        return dJdh
    
    # Entropy loss
    def entropyLoss(self):
        # Need to threshold to prevent division by zero log of zero
        h = copy.copy(self.h) # Threshold h to not modify self.h
        h[h < 1E-5] = 1E-5
        one_minus_h = 1-h
        one_minus_h[one_minus_h < 1E-5] = 1E-5
        J = - np.sum(np.multiply(self.y,np.log(h))+np.multiply((1-self.y),np.log(one_minus_h))) # * is element-wise in python as well, but np.multiply is used here to be explicit         
        return J
    
    # Entropy derivative
    def entropyLossPrime(self):
        # Need to threshold to prevent division by zero log of zero
        h = copy.copy(self.h) # Threshold h to not modify self.h
        h[h < 1E-5] = 1E-5
        one_minus_h = 1-self.h 
        one_minus_h[one_minus_h < 1E-5] = 1E-5
        dJdh = -(self.y/h + (1-self.y)*(-1/(one_minus_h)))
        return dJdh
     
    # Activication function
    def sigmoid(self, z):
        sig = 1/(1+np.exp(-z))
        return sig
    
    # Activation function derivative
    def sigmoidPrime(self, z):
        sigP = np.divide(np.exp(-z),np.square(1+np.exp(-z)))
        return sigP
    
    # Activication function derivative
    def tanhPrime(self, z):
        tanhP = np.tanh(z)
        tanhP = 1-np.square(np.tanh(z))
        return tanhP
    
    # Forward propogation
    def propagateForward(self, X):
        self.w1[:, -1] = 0
        self.w1[-1,-1] = 1
        #X = np.append(X, np.ones((len(X), 1)), axis=1) # Add a 1 for the bias
        self.z1 = np.dot(X, self.w1)
        self.y1 = self.hiddenLayerActFunc(self.z1)
        #self.y1_with_bias = np.append(self.y1, np.ones((len(self.y1), 1)), axis=1) # Add a 1 for the bias
        #self.z2 = np.dot(self.y1_with_bias, self.w2)
        self.z2 = np.dot(self.y1, self.w2)
        self.h = self.outputLayerActFunc(self.z2)
        return 
    
    # Back propagation
    # X = Xtrain, Y = Ytrain = actual digit
    def propagateBackward(self, X, Y):
        # One hot encoding
        self.y = np.zeros((len(Y), 10))
        for i in xrange(0, len(Y)):
            self.y[i, int(Y[i])] = 1
        # dE/dz2 = dy_2/dz_2 * dE/dy_2
        self.dEdz2 = np.multiply(self.outputLayerActFuncPrime(self.z2), self.costPrime())
        # Calculate updates
        self.dEdw2 = np.dot(self.y1.T, self.dEdz2)

        # Don't back propagate the bias
        # dE/dy_1 = dz_2/dy_1 * dEdz_2
        self.dEdy1 = np.dot( self.dEdz2, self.w2.T)
        self.dEdz1 = np.multiply(self.tanhPrime(self.z1), self.dEdy1)
         # Calculate updates
        self.dEdw1 = np.dot(X.T, self.dEdz1)
        return 
    
    # Train the neural network
    def train(self, Xtrain, Ytrain, Xvalid, Yvalid, numiters, num_per_batch):
        start_time = timeit.default_timer()
        training_accuracy_of_last_it = 0.0
        
        # Initial learning rate
        eta = 0.01
        for it_i in xrange(0, numiters):
            # Randomly select data
            sample_ints = random.sample(range(0, Xtrain.shape[0]), num_per_batch)
            X = Xtrain[sample_ints]
            Y = Ytrain[sample_ints]
            self.propagateForward(X)
            self.propagateBackward(X, Y)
            self.y = np.zeros((len(Y), 10))
            for i in xrange(0, len(Y)):
                self.y[i, int(Y[i])] = 1
            

            # Gradient descient updates
            self.w1 = self.w1 - eta * self.dEdw1
            self.w2 = self.w2 - eta * self.dEdw2
            self.w1[:,-1] = 0
            self.w1[-1, -1] = 1
            
            # Calculate accuracy rates and update the learning rate
            if (it_i % 10000) == 0:
                # Calculate the training time
                end_time = timeit.default_timer()
                self.trainingTime.append(end_time - start_time)
                eta = 0.9*eta
                current_training_accuracy = self.calculateAccuracyRate(Xtrain, Ytrain)
                current_validation_accuracy = self.calculateAccuracyRate(Xvalid, Yvalid)
                self.training_accuracy.append(current_training_accuracy)
                self.validation_accuracy.append(current_validation_accuracy)
                print "Iteration number: \t \t", it_i
                print "Time elapsed: \t \t \t", self.trainingTime[-1]
                print "Current training accuracy: \t", current_training_accuracy
                print "Current Validation accuracy: \t", current_validation_accuracy
                training_accuracy_improvement = abs(training_accuracy_of_last_it - current_training_accuracy)
                if (training_accuracy_improvement < 1E-7):
                    return
                training_accuracy_of_last_it =  current_training_accuracy 
        return
    
    # Calculate accuracy
    def calculateAccuracyRate(self, X, Y):
        y_predicted = self.predict(X)
        num_correct = sum(y_predicted == Y)
        accuracy_rate = num_correct/float(Y.shape[0])
        return accuracy_rate
    
    # Used for prediction
    def predict(self, X):
        self.propagateForward(X)
        y_predict = np.zeros((X.shape[0]))
        for i in xrange(0, self.h.shape[0]):
            y_predict[i] = np.argmax(self.h[i,:])
        return y_predict


# Mathematics

For notation, $diag(f(z))$ is a diagonal matrix with $f(x)$ on the diagonal, where $f(x)$ is a function on $x$. Also, $f'(x)$ is the derivative of the function $f(x)$

$W_ij^{(1)}$ and $W_ij^{(2)}$ are  referred to as $w^l$ where $l$ is the layer.


The stochastic gradient equation is:
$$w^{i} = w^{i-1}-\eta \nabla_w J$$

Where $i$ is the iteration number, $\eta$ is the learning rate hyperparameter, and $\nabla_w J$ is the gradient of the loss with respect to $w$. 


The elements of $\nabla_w E$ are $\frac{dJ}{dw_{ij}}$.
The equation for the matrix $\frac{dJ}{dw}$ is the following, with the superscript $l$ indicating the layer number:

$$\frac{dJ}{dw^l} = \frac{dz^l}{dw^l} \frac{dJ}{dz^l}$$


For the $\frac{dz^l}{dw^l}$ term, observe that 
$$z_j = \sum_i {w_{ij} y_i}$$. Taking the derivative of the previous expression, we have that $$\frac{dz^l}{dw^l} = Y^T$$


For the term $\frac{dJ}{dz^l}$, we have

$$\frac{dJ}{dz^l}= \frac{dy^l}{dz^l} \frac{dJ}{dy^l}$$

For the term $\frac{dy^l}{dz^l}$, observe that $y = f(z) = sigmoid(z)$ and $tanh(z)$ for the last layer and for inner layers respectively. Therefore,  $\frac{dy^l}{dz^l} = sigmoid'(z)$ and $tanh'(z)$ respectively. Note that element-wise in the matrix $\frac{dy^l}{dz^l}$, $$sigmoid'(z) = \frac{e^{-z}}{1+e^{-z}}$$ and $$tanh'(z) = 1-tanh^2 (z)$$

To take advantage of matrix calculations,  $\frac{dy^l}{dz^l}$ can be a diagonal matrix with the derivative of each $z$ on the diagonal. (Some coding languages allow element-wise multiplication, in which case $\frac{dy^l}{dz^l}$ can remain a vector with multiplication with other matrices done element-wise)



For the output layer, $\frac{dJ}{dy^l}$ is the derivative of the lost function. 


For squared loss

$$\frac{dJ}{dy} = \frac{d}{dy} 1/2 (y-h)^2 =-(y-h)$$

For entropy

$$\frac{dJ}{dy} = \frac{d}{dy} -[y \ln h + (1-y) \ln (1-h)]= \frac{-y}{h}+\frac{1-y}{1-h}$$

Therefore, for the $w$ between the output layer and last hidden layer, we have

 $$\frac{Ey^l}{dw^l} = \frac{dz^l}{dw^l} \frac{dJ}{dz^l} = \frac{dz^l}{dw^l} \frac{dy^l}{dz^l} \frac{dJ}{dy^l}= -Y^T diag(sigmoid'(z))\frac{dJ}{dy^l} $$

where $\frac{dJ}{dy^l}$ is either the derivative of the squared loss or the derivative of the entropy loss. 

For layers below the output layer, because we have already calculated $\frac{dJ}{dz^l}$ for the last layer, we can recursively write the following
$$\frac{dJ}{dz^{l-1}} = \frac{dy^{l-1}}{dz^{l-1}} \frac{dJ}{dy^{l-1}} =\frac{dy^{l-1}}{dz^{l-1}} \frac{dz^{l}}{dy^{l-1}}\frac{dJ}{dz^{l}} $$

Where $$\frac{dz^{l}}{dy^{l-1}} = w_{l}$$ because $z_j = \sum_i w_{ij} y_i$

Putting this all together, for the $w$ between the hidden and input layers, and between hidden layers, we have

$$\frac{Ey^{l-1}}{dw^{l-1}} = \frac{dz^{l-1}}{dw^{l-1}} \frac{dJ}{dz^{l-1}} = \frac{dz^{l-1}}{dw^{l-1}}\frac{dy^{l-1}}{dz^{l-1}} \frac{dJ}{dy^{l-1}} =\frac{dz^{l-1}}{dw^{l-1}}\frac{dy^{l-1}}{dz^{l-1}} \frac{dz^{l}}{dy^{l-1}}\frac{dJ}{dz^{l}}$$


$$ = Y^{l-1} diag(tanh(z^{l-1})) w^l diag(sigmoid'(z)) \frac{dJ}{dy^l} $$